<a href="https://colab.research.google.com/github/youngmoo/ECES-435/blob/main/Class2-0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction and motivation
This is an example notebook for interactive sound control and visualization with sliders (and pseudo-real-time output) in Colab using [ipywidgets](https://ipywidgets.readthedocs.io/).

* Re-draws figures (using matplotlib) with parameter changes.
* Re-renders and plays sound (using iPython.display) with parameter changes.
* ipywidgets is now installed in Colab (no need to install it as a separate package).

# Initialization

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import IPython.display as ipd
%matplotlib inline
from ipywidgets import *    # Uses interactive, Layout, widgets, Box
from matplotlib import rc

# For JavaScript widget animation (animations will not have sound)
rc('animation', html='jshtml')
rc('animation', embed_limit=2**30)

## My plot style defaults (optional)

In [ ]:
rc('figure', facecolor='#373838')

rc('axes', facecolor='#1e1e1e')
rc('axes', edgecolor='w')
rc('axes', labelcolor='w')
rc('xtick', color='w')
rc('ytick', color='w')

rc('font', family='Liberation Serif')
rc('font', size=20)

# Revisiting complex tones: Interactive additive synthesis
Synthesize, plot, and play a complex tone with 6 sinusoids (a fundametal frequency with 5 additional harmonics)
* The fundamental frequency (pitch) is controled by the `f0` slider (ranging from A1: 55Hz to A5: 880 Hz).
* `a1...a6` are the amplitudes of the harmonics (e.g., `a3` is the weight of frequency `3*f0`).
* Move a slider to start synthesis (plot and sound)

In [ ]:
def complex_tone(f0=220.0, a1=1., a2=0., a3=0., a4=0., a5=0., a6=0.):
    w = 2*np.pi*f0*t    # time samples times 2 * pi * fundamental frequency 
    signal = a1*np.sin(w) + a2*np.sin(2*w) + a3*np.sin(3*w) + a4*np.sin(4*w) + a5*np.sin(5*w) + a6*np.sin(6*w)
    line.set_ydata(signal[:plot_pts])   # Set the line object y-data to the new signal samples
    display(fig)                        # Display the updated figure (otherwise, the plot will not update)
    ipd.display(ipd.Audio(data=signal, rate=fs, autoplay=True))  # 'Display' (and autoplay) the updated sound
    return signal

# Initial parameters
fs = 8000                     # Low sampling rate to minimize computation
dur = 3                       # Duration of synthesized tone (in seconds)
t = np.linspace(0,dur,dur*fs) # Time samples for rendering
plot_pts = 250                # Number of points to plot

# Initialize figure
fig = plt.figure(figsize=(12,4))
ax = plt.axes(xlim=[0, plot_pts/fs], ylim=[-3,3])
line, = ax.plot(t[:plot_pts],np.zeros(plot_pts))
plt.ylabel('Amplitude')
plt.xlabel('Time (sec)')
plt.close()

# Slider layout templates (for horizontal and vertical layouts)
amp_sliders = Layout(display='flex', flex_flow='row', align_items='stretch')
f0_slider = Layout(display='flex', flex_flow='column', align_items='stretch')

labels = ['a1','a2','a3','a4','a5','a6']
# Create a list of amplitude sliders
items = [widgets.FloatSlider(description=label, min=0.0, max=1.0, value=0.0, orientation='vertical') for label in labels]
items[0].value = 1.0

# Use interactive to link the controls and parameters for the complex_tone function
controls = interactive(complex_tone, f0=(55.0,880.0), a1=items[0], a2=items[1], a3=items[2], a4=items[3], a5=items[4], a6=items[5])

# Create the layout (f0 on top, a1...a6 as vertical sliders beneath)
h_slider = Box(children=[controls.children[0]], layout=f0_slider)
v_sliders = Box(children=controls.children[1:], layout=amp_sliders)
display(h_slider)
display(v_sliders)